# Необходимые библиотеки



Для загрузки моделей

In [26]:
!pip install transformers[sentencepiece]

In [27]:
from transformers import BertForMaskedLM, BertTokenizer, RobertaForMaskedLM, RobertaTokenizer, pipeline
import torch

# Корпусы

**Выберите один из датасетов**

* Весёлые истории из жизни

In [49]:
with open("fun_sentences.txt") as file:
  sentences = file.read().splitlines()
  sentences = list(filter(None, sentences))
sentences[:5]

['Вот ну поуговаривала-поуговаривала меня в общем в итоге как-то я так легко так раз ',
 'А денег у нас всё вре всё всего осталось сто рублей.',
 'Ну мы значит по-бырику начинаем сдавать назад.',
 'Думаем если мы развернёмся это будет значит движение по однос по одностороннему движению так нельзя.',
 'И вот как-то раз при мне такое вот ну это дерево своим своей веткой забрало ещё один маленький кактусик.']

In [50]:
with open("fun_masked.txt") as file:
  masked = file.read().splitlines()
  masked = list(filter(None, masked))
masked[:5]

['Вот ну [MASK]-поуговаривала меня в общем в итоге как-то я так легко так раз ',
 'А денег у нас всё [MASK] всё всего осталось сто рублей.',
 'Ну мы значит по-[MASK] начинаем сдавать назад.',
 'Думаем если мы развернёмся это будет значит движение по [MASK] по одностороннему движению так нельзя.',
 'И вот как-то раз при мне такое вот ну это дерево своим своей веткой забрало ещё один маленький [MASK].']

* Рассказы сибиряков о жизни

In [53]:
with open("sib_sentences.txt") as file:
  sentences = file.read().splitlines()
  sentences = list(filter(None, sentences))
sentences[:5]

['Когда я приехала мы с друзьями решили сходить в горы В в горы мы ходили у нас место называется Черкосай туда очень далеко идти вот и долго надо ехать вот сначала мы долго ехали на машине потом кстати ну ну нас высадили и мы пошли пешком в горы мы очень долго шли Часов наверно пять.',
 'Вот это меня очень впечатлило.',
 'Ну в конце было более позитивно в конце были два крокодильчика лежащие друг на друге просто греющиеся под лампой бы света.',
 'Нашёл работу бильярдистом то есть смотреть з за столами смотреть чтобы правила не нарушали вот сам хочу научиться кайфово в бильярд играть.',
 'Всё мужики опять прибежали «Ты говорит вообще здесь охренел!']

In [54]:
with open("sib_masked.txt") as file:
  masked = file.read().splitlines()
  masked = list(filter(None, masked))
masked[:5]

['Когда я приехала мы с друзьями решили сходить в горы В в горы мы ходили у нас место называется [MASK] туда очень далеко идти вот и долго надо ехать вот сначала мы долго ехали на машине потом кстати ну ну нас высадили и мы пошли пешком в горы мы очень долго шли Часов наверно пять.',
 'Вот это меня очень [MASK].',
 'Ну в конце было более позитивно в конце были два [MASK] лежащие друг на друге просто греющиеся под лампой бы света.',
 'Нашёл работу бильярдистом то есть смотреть з за столами смотреть чтобы правила не нарушали вот сам хочу научиться [MASK] в бильярд играть.',
 'Всё мужики опять прибежали «Ты говорит вообще здесь [MASK]!']

# Функция проверки предсказывания

In [30]:
def predict_mask(unmasker, sentences, masked):
  for i in range(len(masked)):
    print(f"\n{sentences[i]}\n{masked[i]}")
    for i in unmasker(masked[i]):
      score = '{:.2f}'.format(float(i['score']) * 100)
      token = "".join(i['token_str'].split(" "))
      print(f"{score}% \"{token}\"\n{i['sequence']}")

# Сравнение моделей

## `ruBert-large`

In [55]:
model=BertForMaskedLM.from_pretrained('sberbank-ai/ruBert-large')
tokenizer=BertTokenizer.from_pretrained('sberbank-ai/ruBert-large', do_lower_case=False)
unmasker = pipeline('fill-mask', model=model,tokenizer=tokenizer)

Some weights of the model checkpoint at sberbank-ai/ruBert-large were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [56]:
predict_mask(unmasker, sentences, masked)


Когда я приехала мы с друзьями решили сходить в горы В в горы мы ходили у нас место называется Черкосай туда очень далеко идти вот и долго надо ехать вот сначала мы долго ехали на машине потом кстати ну ну нас высадили и мы пошли пешком в горы мы очень долго шли Часов наверно пять.
Когда я приехала мы с друзьями решили сходить в горы В в горы мы ходили у нас место называется [MASK] туда очень далеко идти вот и долго надо ехать вот сначала мы долго ехали на машине потом кстати ну ну нас высадили и мы пошли пешком в горы мы очень долго шли Часов наверно пять.
4.88% "Эльбрус"
Когда я приехала мы с друзьями решили сходить в горы В в горы мы ходили у нас место называется Эльбрус туда очень далеко идти вот и долго надо ехать вот сначала мы долго ехали на машине потом кстати ну ну нас высадили и мы пошли пешком в горы мы очень долго шли Часов наверно пять.
4.58% "а"
Когда я приехала мы с друзьями решили сходить в горы В в горы мы ходили у нас место называется а туда очень далеко идти вот и д

## `ruRoberta-large`

In [57]:
tokenizer=RobertaTokenizer.from_pretrained('sberbank-ai/ruRoberta-large')
model=RobertaForMaskedLM.from_pretrained('sberbank-ai/ruRoberta-large')
unmasker = pipeline('fill-mask', model=model,tokenizer=tokenizer)

Оказалось, что у ruRoberta маска немного другого вида

In [58]:
masked_for_roberta = []
for i in masked:
  new_mask_sentence = i[:i.find("[MASK]")] + "<mask>" + i[i.find("[MASK]")+6:]
  masked_for_roberta.append(new_mask_sentence)

In [59]:
predict_mask(unmasker, sentences, masked_for_roberta)


Когда я приехала мы с друзьями решили сходить в горы В в горы мы ходили у нас место называется Черкосай туда очень далеко идти вот и долго надо ехать вот сначала мы долго ехали на машине потом кстати ну ну нас высадили и мы пошли пешком в горы мы очень долго шли Часов наверно пять.
Когда я приехала мы с друзьями решили сходить в горы В в горы мы ходили у нас место называется <mask> туда очень далеко идти вот и долго надо ехать вот сначала мы долго ехали на машине потом кстати ну ну нас высадили и мы пошли пешком в горы мы очень долго шли Часов наверно пять.
5.92% ","
Когда я приехала мы с друзьями решили сходить в горы В в горы мы ходили у нас место называется, туда очень далеко идти вот и долго надо ехать вот сначала мы долго ехали на машине потом кстати ну ну нас высадили и мы пошли пешком в горы мы очень долго шли Часов наверно пять.
3.90% "это"
Когда я приехала мы с друзьями решили сходить в горы В в горы мы ходили у нас место называется это туда очень далеко идти вот и долго надо